# 1. Imports

In [0]:
%run /Workspace/Users/matheusfelipebasso@hotmail.com/Datum/utils/olist/SAFE_SAVE_SILVER

In [0]:
from pyspark.sql.functions import datediff, col

---
# 2. Leitura

In [0]:
df_orders = spark.table('datum.silver.olist_orders')

In [0]:
df_orders = (df_orders.
             withColumn('delivery_diff_promissed',
                        datediff(
                            col('order_delivered_customer_date'),
                            col('order_estimated_delivery_date')
                                 )).
             select('order_id', 'customer_id', 'delivery_diff_promissed').
             where(col('delivery_diff_promissed').isNotNull())
             )


Significado da nova coluna

| delivery_diff_promissed | significado |
|-------------------------|-------------|
| < 0 | Entrega realizada e antes do prazo|
| = 0 | Entrega realizada no prazo |
| > 0 | Entrega realizada depois do prazo |
| `None` | Pedido não entregue |

Também foi realizado um `where` removendo onde não temos as datas pois em caso de uso da coluna o recomendado seria um `LEFT JOIN`, com isso os campos nulos apenas seriam repetidos, assim realizando um filtro para eliminar esses nulos reduzimos o tamanho do DF e diminuindo custos com armazenagem. De certa forma a tabela irá agir como uma `dim` table para a `orders` que seria como a `fact`

In [0]:
df_orders.printSchema()

root
 |-- order_id: string (nullable = false)
 |-- customer_id: string (nullable = false)
 |-- delivery_diff_promissed: integer (nullable = true)



---
# 4. Delta Lake

Utilizando `format("delta")` para aproveitar as vantagens da estrutura.

In [0]:
olist_silver_delta_saving(df=df_orders, table_name='olist_orders_datediff')

Sucesso para para a tabela "datum.silver.olist_orders_datediff"
Tempo de Execução: 00:00:07


In [0]:
del df_orders